In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import load_model
import numpy as np

In [49]:
import pickle

# Load all the models
model=load_model('model.h5')

with open('label_encoder.pkl','rb') as file:
    label_encoder=pickle.load(file)
    
with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)
    
with open('one_hot_encoder.pkl','rb') as file:
    one_hot_encoder=pickle.load(file)
    
input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':10000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

one_hot_encoded = one_hot_encoder.transform([[input_data['Geography']]]).toarray()
one_hot_encoder_df = pd.DataFrame(one_hot_encoded, columns=['Geography_France','Geography_Germany','Geography_Spain'])
one_hot_encoder_df

c:\Users\parkh\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [51]:
input_df=pd.DataFrame(input_data,index=[0])

input_df['Gender']=label_encoder.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,10000,2,1,1,50000


In [54]:
new_data=pd.concat([input_df,one_hot_encoder_df],axis=1).drop(columns=['Geography'],axis=1)
new_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,10000,2,1,1,50000,1.0,0.0,0.0


In [55]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)
    
scaled_data=scaler.transform(new_data)
scaled_data

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -1.05836066,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [56]:
prediction=model.predict(scaled_data)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step


array([[0.19309571]], dtype=float32)

In [58]:
prediction_proba=prediction[0][0]
if prediction_proba > 0.5:
    prediction_label = 'The Customer is likely to Churn'
else:
    prediction_label = 'The Customer is not likely to Churn'  

print(f'Prediction Probability: {prediction_proba}')
print(prediction_label)

Prediction Probability: 0.19309571385383606
The Customer is not likely to Churn
